<a href="https://colab.research.google.com/github/tahmidmir/100-Questions-and-Answers-for-Data-Science-Interviews-2025-Edition-/blob/main/Climate_Data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
import time

cities = ["Malindi","Vihiga","Machakos","Kisii","Ngong","Mumias","Thika","Nyeri","Kakamega","Wajir","Ukunda","Nandi Hills","Narok","Embu","Kitale","Wundanyi","El Wak","Wote","Kimilili","Bungoma","Isiolo","Meru","Webuye","Iten","Homa Bay","Rumuruti","Maralal","Busia","Mandera","Kericho","Kitui","Lamu","Kajiado","Kapsabet","Marsabit","Lodwar","Kerugoya","Kwale","Hola","Mwatate","Kabarnet","Migori","Nyamira","Sotik Post","Murang’a","Siaya","Kapenguria","Ol Kalou"]
latitude = [-3.2236, 0.05, -1.5167, -0.6833, -1.3667, 0.3333, -1.0396, -0.4167, 0.2822, 1.75, -4.2875, 0.1003, -1.0833, -0.5389, 1.0167, -3.3983, 2.8028, -1.7833, 0.7833, 0.5667, 0.35, 0.05, 0.6167, 0.6731, -0.5167, 0.26, 1.1, 0.4633, 3.9167, -0.3692, -1.3667, -2.2694, -1.85, 0.3333, 2.3333, 3.1167, -0.5, -4.1744, -1.5, -3.5047, 0.494, -1.0634, -0.521, -0.7813, -0.7167, 0.0667, 1.2333, -0.273]
longitude = [40.13, 34.725, 37.2667, 34.7667, 36.6333, 34.4833, 37.09, 36.95, 34.754, 40.05, 39.5661, 35.1764, 35.8667, 37.4583, 35, 38.3603, 40.9275, 37.6333, 34.7167, 34.5667, 37.5833, 37.65, 34.7667, 35.5083, 34.45, 36.5363, 36.7, 34.1053, 41.8333, 35.2839, 38.0167, 40.9022, 36.7833, 35.1667, 37.9833, 35.6, 37.2833, 39.4603, 40.0333, 38.3778, 35.744, 34.4731, 34.914, 35.3416, 37.15, 34.2833, 35.1167, 36.378]
start_date = "2012-01-01"
end_date = "2024-08-13"

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"

if len(cities) == len(latitude) and len(cities) == len(longitude):
    for i in range(len(cities)):
        params = {
          "latitude": latitude[i],
          "longitude": longitude[i],
          "start_date": start_date,
          "end_date": end_date,
          "timezone": "Africa/Nairobi",
          "daily": ["weather_code", "temperature_2m_max", "temperature_2m_min", "temperature_2m_mean", "apparent_temperature_max", "apparent_temperature_min", "apparent_temperature_mean", "sunrise", "sunset", "daylight_duration", "sunshine_duration", "precipitation_sum", "rain_sum", "snowfall_sum", "precipitation_hours", "wind_speed_10m_max", "wind_gusts_10m_max", "wind_direction_10m_dominant", "shortwave_radiation_sum", "et0_fao_evapotranspiration"]
        }
        responses = openmeteo.weather_api(url, params=params)

        # Process first location. Add a for-loop for multiple locations or weather models
        response = responses[0]
        print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
        print(f"Elevation {response.Elevation()} m asl")
        print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
        print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

        # Process daily data. The order of variables needs to be the same as requested.
        daily = response.Daily()
        daily_weather_code = daily.Variables(0).ValuesAsNumpy()
        daily_temperature_2m_max = daily.Variables(1).ValuesAsNumpy()
        daily_temperature_2m_min = daily.Variables(2).ValuesAsNumpy()
        daily_temperature_2m_mean = daily.Variables(3).ValuesAsNumpy()
        daily_apparent_temperature_max = daily.Variables(4).ValuesAsNumpy()
        daily_apparent_temperature_min = daily.Variables(5).ValuesAsNumpy()
        daily_apparent_temperature_mean = daily.Variables(6).ValuesAsNumpy()
        daily_sunrise = daily.Variables(7).ValuesAsNumpy()
        daily_sunset = daily.Variables(8).ValuesAsNumpy()
        daily_daylight_duration = daily.Variables(9).ValuesAsNumpy()
        daily_sunshine_duration = daily.Variables(10).ValuesAsNumpy()
        daily_precipitation_sum = daily.Variables(11).ValuesAsNumpy()
        daily_rain_sum = daily.Variables(12).ValuesAsNumpy()
        daily_snowfall_sum = daily.Variables(13).ValuesAsNumpy()
        daily_precipitation_hours = daily.Variables(14).ValuesAsNumpy()
        daily_wind_speed_10m_max = daily.Variables(15).ValuesAsNumpy()
        daily_wind_gusts_10m_max = daily.Variables(16).ValuesAsNumpy()
        daily_wind_direction_10m_dominant = daily.Variables(17).ValuesAsNumpy()
        daily_shortwave_radiation_sum = daily.Variables(18).ValuesAsNumpy()
        daily_et0_fao_evapotranspiration = daily.Variables(19).ValuesAsNumpy()

        daily_data = {"date": pd.date_range(
          start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
          end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
          freq = pd.Timedelta(seconds = daily.Interval()),
          inclusive = "left"
        )}
        daily_data["weather_code"] = daily_weather_code
        daily_data["temperature_2m_max"] = daily_temperature_2m_max
        daily_data["temperature_2m_min"] = daily_temperature_2m_min
        daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
        daily_data["apparent_temperature_max"] = daily_apparent_temperature_max
        daily_data["apparent_temperature_min"] = daily_apparent_temperature_min
        daily_data["apparent_temperature_mean"] = daily_apparent_temperature_mean
        daily_data["sunrise"] = daily_sunrise
        daily_data["sunset"] = daily_sunset
        daily_data["daylight_duration"] = daily_daylight_duration
        daily_data["sunshine_duration"] = daily_sunshine_duration
        daily_data["precipitation_sum"] = daily_precipitation_sum
        daily_data["rain_sum"] = daily_rain_sum
        daily_data["snowfall_sum"] = daily_snowfall_sum
        daily_data["precipitation_hours"] = daily_precipitation_hours
        daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
        daily_data["wind_gusts_10m_max"] = daily_wind_gusts_10m_max
        daily_data["wind_direction_10m_dominant"] = daily_wind_direction_10m_dominant
        daily_data["shortwave_radiation_sum"] = daily_shortwave_radiation_sum
        daily_data["et0_fao_evapotranspiration"] = daily_et0_fao_evapotranspiration

        daily_dataframe = pd.DataFrame(data = daily_data)
        filename = f"{cities[i]}_daily_weather_{start_date}_to_{end_date}.csv"
        daily_dataframe.to_csv(filename, index=False)
        print(f"Saved data to {filename}")

        time.sleep(60)

else:
    print("The number of cities, latitudes, and longitudes should be the same.")


Coordinates -3.198594093322754°N 40.09685516357422°E
Elevation 0.0 m asl
Timezone b'Africa/Nairobi' b'EAT'
Timezone difference to GMT+0 10800 s
Saved data to Malindi_daily_weather_2012-01-01_to_2024-08-13.csv
Coordinates 0.035149384289979935°N 34.766357421875°E
Elevation 1614.0 m asl
Timezone b'Africa/Nairobi' b'EAT'
Timezone difference to GMT+0 10800 s
Saved data to Vihiga_daily_weather_2012-01-01_to_2024-08-13.csv
Coordinates -1.5114235877990723°N 37.268409729003906°E
Elevation 1628.0 m asl
Timezone b'Africa/Nairobi' b'EAT'
Timezone difference to GMT+0 10800 s
Saved data to Machakos_daily_weather_2012-01-01_to_2024-08-13.csv
Coordinates -0.6678382754325867°N 34.72941207885742°E
Elevation 1685.0 m asl
Timezone b'Africa/Nairobi' b'EAT'
Timezone difference to GMT+0 10800 s
Saved data to Kisii_daily_weather_2012-01-01_to_2024-08-13.csv
Coordinates -1.370826005935669°N 36.640316009521484°E
Elevation 2088.0 m asl
Timezone b'Africa/Nairobi' b'EAT'
Timezone difference to GMT+0 10800 s
Saved 

OpenMeteoRequestsError: {'error': True, 'reason': 'Hourly API request limit exceeded. Please try again in the next hour.'}